In [2]:
import spacy
nlp = spacy.load("en_core_web_md")

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
import common as c
raw_tag = '{BAGGU,groupbycolor,"home goods",Textiles}'
raw_title = 'Reusable Cloth Set / Backyard Fruit'
raw_product_type = 'textiles'
#line 31 example
raw_product_description = 'You may find yourself in a sticky situation. \
    You may need to secure that mushroom you came across in the woods. \
    You may need a tablecloth for a very small table. \
    You may have a runny nose or cold neck or be having a bad hair day and just want to cover it up. \
    You may ask yourself why you left home without a square of fabric. \
    You may have just found the solution — 20 square inches of pure possibility.\
    Details:Set of threeMeasures 20'' H x 20'' W100% Organic Cotton\
    About the Brand:Founded in 2007, BAGGU set out to create a reusable bag that was as functional as it was adorable. \
    Today their goal is to make every bag you need for your every day life. \
    They have stuck with their mission of creating useful products that are made with you and the planet in mind. \
    BAGGU is manufactured ethically and environmentally responsibly in China.'
#pre-process the tag/title/product_type
content_1 = c.clean_tags_text(raw_tag, raw_title, raw_product_type)
#pre-process the production_description
content_2 = c.clean_product_description(raw_product_description)


[(cotopaxi, 'cotopaxi', 'PROPN'), (outerwear, 'outerwear', 'VERB'), (womens, 'women', 'NOUN'), ( , ' ', 'SPACE'), (cotopaxi, 'cotopaxi', 'NOUN'), (women, 'woman', 'NOUN'), ('s, "'s", 'PART'), (teca, 'teca', 'NOUN'), (half, 'half', 'ADJ'), (-, '-', 'PUNCT'), (zip, 'zip', 'NOUN'), (windbreaker, 'windbreaker', 'NOUN'), (postcard, 'postcard', 'PROPN'), ( , ' ', 'SPACE'), (jackets, 'jacket', 'NOUN')]
jacket


In [4]:
type_matcher = c.create_patterns_matcher()
doc = nlp(content_2 + content_1)

for idx, start, end in type_matcher(doc):
    print(doc[start:end])

bag
bag


#### 1. First we are trainning model to detect all products belonging to 'TOPS'
(TODO: overlap type in matcher eg. "t-shirt" -> 'shirt' and 't-shirt') 

In [5]:
def parse_train_data(text):
    doc = nlp(text)
    #ignore for now 
    detections = [(doc[start:end].start_char, doc[start:end].end_char, 'TOPS') for idx, start, end in type_matcher(doc)]
    
    return (doc.text, {'entities': detections})

#parse_train_data("I like my jacket")
parse_train_data("These camisole and T-shirt and are so good. I did have similar pattern jacket which is so fancy")

('These camisole and T-shirt and are so good. I did have similar pattern jacket which is so fancy',
 {'entities': [(6, 14, 'TOPS'),
   (19, 26, 'TOPS'),
   (21, 26, 'TOPS'),
   (71, 77, 'TOPS')]})

#### 2. Next step: We need to operate sample dataset to seperate the 'product_type_number == 2' to create classifier of 'TOPS' trainning model


In [6]:
import pandas as pd
text_type = pd.read_csv('sample_v1.csv', usecols=["title", "tags", "product_type", "body_html", "product_type_number"])

tops_df = text_type.loc[text_type['product_type_number'] == '2'].reset_index()
#pd.concat([pd.DataFrame([i], columns=['label']) for i in range(70)])

tops_df.insert(len(tops_df.columns), 'label', 1, allow_duplicates=True)
print("columns", len(tops_df.columns))
#prediction_1 is based on tags + title + product_type
tops_df.insert(7, 'prediction_1', 0, allow_duplicates=True)
#prediction_2 is based on product_description(body_html)
tops_df.insert(len(tops_df.columns), 'prediction_2', 0, allow_duplicates=True)
#prdiction_3 is based on prediction1 + prdiction 2 (all words) a & b
tops_df.insert(len(tops_df.columns), 'prediction', 0, allow_duplicates=True)
#fill all empty cells 
tops_df.fillna("Not mention", inplace=True)

columns 7


In [7]:
#loop all rows with each row 'title', 'tags' and 'product_type' to create new column called 'title+tag'
type_matcher = c.create_patterns_matcher()
errorcount = 0
for i in range(len(tops_df)):
    content_2_raw = ''
    content_1_raw = ''
    content_raw = ''
    try:
        content_1_raw = c.clean_tags_text(tops_df.loc[i, 'tags'], tops_df.loc[i, 'title'], tops_df.loc[i, 'product_type'])
        
        content_2_raw = c.clean_product_description(tops_df.loc[i, 'body_html'])
        
        content_raw = content_1_raw + ', ' + content_2_raw
        
    except:
        print("line type error ",i)
    
    content = nlp(content_raw)
    content_1 = nlp(content_1_raw)
    content_2 = nlp(content_2_raw)
    
    if len(type_matcher(content_1)) > 0:
        tops_df.loc[i, 'prediction_1'] = 1

        
    if len(type_matcher(content_2)) > 0:
        tops_df.loc[i, 'prediction_2'] = 1
    
    if len(type_matcher(content)) > 0:
        tops_df.loc[i, 'prediction'] = 1
    else:
        errorcount += 1
        print(errorcount, content_1_raw)
        print(errorcount, content_2_raw)
    





#### 3. After every column using type_matcher, we update the 'prediction' column.


In [12]:
tops_df.rename(columns={"index": "index_in_original_sample"}, inplace=True)
tops_df.to_csv('TOPS_result.csv')

#### 4. Build TRIAN_DATA for 'tops'

In [13]:
tops_df.loc[tops_df['prediction'] == 0]

,index_in_original_sample,title,product_type,tags,product_type_number,body_html,label,prediction_1,prediction_2,prediction
